<a href="https://colab.research.google.com/github/GuyFarb/Cars-Project/blob/main/cnn7.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# 🔸 חיבור לגוגל דרייב
from google.colab import drive
drive.mount('/content/drive')

# 🔸 ייבוא ספריות
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns
from sklearn.metrics import classification_report, accuracy_score, confusion_matrix

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, GlobalAveragePooling2D, Dense, Dropout, BatchNormalization
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau

# 🔸 פונקציית פירוק TFRecord
def parse_example(example_proto):
    feature_description = {
        'image': tf.io.FixedLenFeature([], tf.string),
        'label': tf.io.FixedLenFeature([], tf.int64),
    }
    parsed = tf.io.parse_single_example(example_proto, feature_description)
    image = tf.image.decode_jpeg(parsed['image'], channels=3)
    image = tf.image.resize(image, [224, 224])
    image = tf.cast(image, tf.float32) / 255.0
    label = parsed['label']
    return image, label

# 🔸 טעינת סט האימון
train_path = "/content/drive/MyDrive/Colab Notebooks/cars196_train.tfrecord"
train_dataset = tf.data.TFRecordDataset(train_path).map(parse_example)

images_train, labels_train = [], []
for img, lbl in train_dataset:
    images_train.append(img.numpy())
    labels_train.append(lbl.numpy())

images_train = np.array(images_train)
labels_train = np.array(labels_train)
labels_train_cat = to_categorical(labels_train, num_classes=196)

print(f"✅ Train Images: {images_train.shape}, Labels: {labels_train.shape}")

# 🔸 CNN קליל ומהיר
model = Sequential([
    Conv2D(32, (3,3), activation='relu', padding='same', input_shape=(224,224,3)),
    BatchNormalization(),
    MaxPooling2D(),

    Conv2D(64, (3,3), activation='relu', padding='same'),
    BatchNormalization(),
    MaxPooling2D(),

    Conv2D(128, (3,3), activation='relu', padding='same'),
    BatchNormalization(),
    MaxPooling2D(),

    GlobalAveragePooling2D(),
    Dense(256, activation='relu'),
    Dropout(0.4),
    Dense(196, activation='softmax')
])

model.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy', 'top_k_categorical_accuracy'])

model.summary()

# 🔸 Data Augmentation
datagen = ImageDataGenerator(
    rotation_range=15,
    zoom_range=0.1,
    width_shift_range=0.1,
    height_shift_range=0.1,
    horizontal_flip=True,
    validation_split=0.2
)

# 🔸 Callbacks
callbacks = [
    EarlyStopping(monitor='val_accuracy', patience=3, restore_best_weights=True, mode='max', verbose=1),
    ReduceLROnPlateau(monitor='val_accuracy', factor=0.5, patience=2, mode='max', verbose=1)
]

# 🔸 אימון מהיר יותר עם batch גדול
history = model.fit(
    datagen.flow(images_train, labels_train_cat, subset='training', batch_size=128),
    validation_data=datagen.flow(images_train, labels_train_cat, subset='validation', batch_size=128),
    epochs=15,
    callbacks=callbacks,
    verbose=1
)

# 🔸 שמירת המודל לאחר אימון
model.save("cnn_model_faster_after_train.h5")
print("✅ המודל נשמר לאחר אימון!")

# ===============================================
# 🔸 🔹 🔸 שלב ה-Test 🔸 🔹 🔸
# ===============================================
test_path = "/content/drive/MyDrive/Colab Notebooks/cars196_test.tfrecord"
test_dataset = tf.data.TFRecordDataset(test_path).map(parse_example)

images_test, labels_test = [], []
for img, lbl in test_dataset:
    images_test.append(img.numpy())
    labels_test.append(lbl.numpy())

images_test = np.array(images_test)
labels_test = np.array(labels_test)

# תחזית
predictions_test = model.predict(images_test, batch_size=128)
y_pred = np.argmax(predictions_test, axis=1)
y_true = labels_test

# דוח ביצועים
report = classification_report(y_true, y_pred, digits=4, output_dict=True)
report_df = pd.DataFrame(report).transpose()
report_df.to_csv("cnn_fast_classification_report_test.csv")
print(report_df.head(50))

# דיוק סופי
acc = accuracy_score(y_true, y_pred)
print(f"\n🎯 דיוק סופי על סט ה-Test: {acc:.4f}")

# שמירת המודל לאחר Test
model.save("cnn_model_faster_after_test.h5")
print("✅ המודל נשמר לאחר Test!")

# 🔸 גרפים
plt.figure(figsize=(12,5))
plt.subplot(1,2,1)
plt.plot(history.history['accuracy'], label='Train Accuracy')
plt.plot(history.history['val_accuracy'], label='Validation Accuracy')
plt.title('📊 Accuracy לאורך Epochs')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.legend()

plt.subplot(1,2,2)
plt.plot(history.history['loss'], label='Train Loss')
plt.plot(history.history['val_loss'], label='Validation Loss')
plt.title('📉 Loss לאורך Epochs')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.legend()
plt.show()

# 🔸 Confusion Matrix
cm = confusion_matrix(y_true, y_pred)
plt.figure(figsize=(14,12))
sns.heatmap(cm, cmap="Blues", square=False, xticklabels=False, yticklabels=False)
plt.title("🌀 Confusion Matrix על סט ה-Test")
plt.xlabel("Predicted")
plt.ylabel("True")
plt.show()


Mounted at /content/drive
✅ Train Images: (8144, 224, 224, 3), Labels: (8144,)


/usr/local/lib/python3.11/dist-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ conv2d (Conv2D)                      │ (None, 224, 224, 32)        │             896 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ batch_normalization                  │ (None, 224, 224, 32)        │             128 │
│ (BatchNormalization)                 │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling2d (MaxPooling2D)         │ (None, 112, 112, 32)        │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d_1 (Conv2D)                    │ (None, 112, 112, 64)        │          18,496 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ batch_normalization_1                │ (None, 112, 112, 64)        │             256 │
│ (BatchNormalization)                 │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling2d_1 (MaxPooling2D)       │ (None, 56, 56, 64)          │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d_2 (Conv2D)                    │ (None, 56, 56, 128)         │          73,856 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ batch_normalization_2                │ (None, 56, 56, 128)         │             512 │
│ (BatchNormalization)                 │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling2d_2 (MaxPooling2D)       │ (None, 28, 28, 128)         │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ global_average_pooling2d             │ (None, 128)                 │               0 │
│ (GlobalAveragePooling2D)             │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense (Dense)                        │ (None, 256)                 │          33,024 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout (Dropout)                    │ (None, 256)                 │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_1 (Dense)                      │ (None, 196)                 │          50,372 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 177,540 (693.52 KB)

 Trainable params: 177,092 (691.77 KB)

 Non-trainable params: 448 (1.75 KB)

/usr/local/lib/python3.11/dist-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


Epoch 1/15
51/51 ━━━━━━━━━━━━━━━━━━━━ 137s 2s/step - accuracy: 0.0066 - loss: 5.3315 - top_k_categorical_accuracy: 0.0287 - val_accuracy: 0.0012 - val_loss: 5.3231 - val_top_k_categorical_accuracy: 0.0184 - learning_rate: 0.0010
Epoch 2/15
51/51 ━━━━━━━━━━━━━━━━━━━━ 101s 2s/step - accuracy: 0.0230 - loss: 5.1779 - top_k_categorical_accuracy: 0.0709 - val_accuracy: 0.0037 - val_loss: 5.5617 - val_top_k_categorical_accuracy: 0.0184 - learning_rate: 0.0010
Epoch 3/15
51/51 ━━━━━━━━━━━━━━━━━━━━ 100s 2s/step - accuracy: 0.0188 - loss: 5.0957 - top_k_categorical_accuracy: 0.0869 - val_accuracy: 0.0037 - val_loss: 5.9149 - val_top_k_categorical_accuracy: 0.0276 - learning_rate: 0.0010
Epoch 4/15
51/51 ━━━━━━━━━━━━━━━━━━━━ 98s 2s/step - accuracy: 0.0232 - loss: 5.0260 - top_k_categorical_accuracy: 0.0923 - val_accuracy: 0.0074 - val_loss: 6.1540 - val_top_k_categorical_accuracy: 0.0258 - learning_rate: 0.0010
Epoch 5/15
51/51 ━━━━━━━━━━━━━━━━━━━━ 101s 2s/step - accuracy: 0.0379 - loss: 4.9164 

✅ המודל נשמר לאחר אימון!
